In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize 
import itertools

data = pd.read_feather("dataFrame.feather")
data["category"] = pd.Categorical(data["category"])
print(data.shape)
X = data["text"]
Y = pd.get_dummies(data["category"])
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.1)


(2225, 3)


In [10]:
X_unique = [word_tokenize(text.lower()) for text in train_X]
X_unique = pd.value_counts(list(itertools.chain.from_iterable(X_unique)))
X_unique = X_unique[X_unique>9]
X_unique = list(X_unique.index)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(vocabulary=X_unique) 
X_train_tfidf = tfidf.fit_transform(train_X)
X_test_tfidf = tfidf.transform(test_X)

Y_train =  np.array(train_Y).argmax(1)
Y_test =  np.array(test_Y).argmax(1)

In [12]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
np.max(X_train_tfidf.todense())

(2002, 6566)
(223, 6566)


0.8139427842511371

In [13]:
from sklearn.naive_bayes import GaussianNB
import time
start_time = time.time()
clf = GaussianNB()
clf.fit(X_train_tfidf.todense(), Y_train)
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.1649770736694336 seconds ---


In [14]:
np.sum(clf.predict(X_train_tfidf.todense())==Y_train)/len(Y_train)

1.0

In [15]:
np.sum(clf.predict(X_test_tfidf.todense())==Y_test)/len(Y_test)

0.9013452914798207